## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,42.0970,-79.2353,43.72,86,1,4.32,clear sky
1,1,Dikson,RU,73.5069,80.5464,14.52,91,90,16.33,overcast clouds
2,2,Qaanaaq,GL,77.4840,-69.3632,16.05,76,100,6.22,overcast clouds
3,3,La Palma,US,33.8464,-118.0467,66.92,84,75,5.01,broken clouds
4,4,Shostka,UA,51.8630,33.4698,50.54,74,99,11.14,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temp_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
temp_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Isangel,VU,-19.5500,169.2667,77.27,60,31,16.40,scattered clouds
13,13,George Town,MY,5.4112,100.3354,89.55,66,40,6.91,light rain
17,17,Harper,LR,4.3750,-7.7169,80.10,84,100,6.42,overcast clouds
21,21,Inverell,AU,-29.7833,151.1167,73.11,29,0,3.00,clear sky
23,23,Sabang,ID,5.8933,95.3214,82.92,73,99,5.19,overcast clouds
24,24,Puerto Ayora,EC,-0.7393,-90.3518,71.40,93,21,8.19,few clouds
27,27,Port Blair,IN,11.6667,92.7500,87.84,74,75,13.80,broken clouds
30,30,Lakatoro,VU,-16.0999,167.4164,81.57,64,48,18.43,scattered clouds
31,31,Vaini,TO,-21.2000,-175.2000,77.16,78,40,8.05,scattered clouds
33,33,Vao,NC,-22.6667,167.4833,74.37,71,52,18.61,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
temp_cities_df.count()

City_ID                281
City                   281
Country                281
Lat                    281
Lng                    281
Max Temp               281
Humidity               281
Cloudiness             281
Wind Speed             281
Current Description    281
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = temp_cities_df.dropna()
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Isangel,VU,-19.5500,169.2667,77.27,60,31,16.40,scattered clouds
13,13,George Town,MY,5.4112,100.3354,89.55,66,40,6.91,light rain
17,17,Harper,LR,4.3750,-7.7169,80.10,84,100,6.42,overcast clouds
21,21,Inverell,AU,-29.7833,151.1167,73.11,29,0,3.00,clear sky
23,23,Sabang,ID,5.8933,95.3214,82.92,73,99,5.19,overcast clouds
24,24,Puerto Ayora,EC,-0.7393,-90.3518,71.40,93,21,8.19,few clouds
27,27,Port Blair,IN,11.6667,92.7500,87.84,74,75,13.80,broken clouds
30,30,Lakatoro,VU,-16.0999,167.4164,81.57,64,48,18.43,scattered clouds
31,31,Vaini,TO,-21.2000,-175.2000,77.16,78,40,8.05,scattered clouds
33,33,Vao,NC,-22.6667,167.4833,74.37,71,52,18.61,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Isangel,VU,77.27,scattered clouds,-19.5500,169.2667,
13,George Town,MY,89.55,light rain,5.4112,100.3354,
17,Harper,LR,80.10,overcast clouds,4.3750,-7.7169,
21,Inverell,AU,73.11,clear sky,-29.7833,151.1167,
23,Sabang,ID,82.92,overcast clouds,5.8933,95.3214,
24,Puerto Ayora,EC,71.40,few clouds,-0.7393,-90.3518,
27,Port Blair,IN,87.84,broken clouds,11.6667,92.7500,
30,Lakatoro,VU,81.57,scattered clouds,-16.0999,167.4164,
31,Vaini,TO,77.16,scattered clouds,-21.2000,-175.2000,
33,Vao,NC,74.37,broken clouds,-22.6667,167.4833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Isangel,VU,77.27,scattered clouds,-19.5500,169.2667,Tanna Lodge
13,George Town,MY,89.55,light rain,5.4112,100.3354,Cititel Penang
17,Harper,LR,80.10,overcast clouds,4.3750,-7.7169,Screensaver
21,Inverell,AU,73.11,clear sky,-29.7833,151.1167,Sapphire City Motor Inn
23,Sabang,ID,82.92,overcast clouds,5.8933,95.3214,Hotel Citra
...,...,...,...,...,...,...,...
693,Hirado,JP,70.39,few clouds,33.3597,129.5531,Samson Hotel
694,Comodoro Rivadavia,AR,71.46,clear sky,-45.8667,-67.5000,Austral Hotel
697,Koumac,NC,80.53,few clouds,-20.5667,164.2833,Le Monitel
699,Quchan,IR,73.49,overcast clouds,37.1060,58.5095,گلیم دستبافت کیمیایی


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 

locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
current_temp = clean_hotel_df["Current Description"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))